In [1]:
import hyperspy.api as hs
import plotly.graph_objects as go
import numpy as np

from utils.get_raw_data import locate_raw_data, get_multiple_data_arrays
from helper_files.plotting import plotly_plot
from helper_files.gaussian_fitting import gaussian, n_gaussians, fit_n_peaks_to_gaussian, area_under_peak
from helper_files.calibration import calibrate_channel_width_two_peaks, channel_to_keV
from helper_files.spectrum_dict import init_known_spectrum
from helper_files.peak_ratios import  n_gaussians_and_m_order_bg, make_fit


emsa = locate_raw_data()

In [2]:
s = init_known_spectrum(
    name="GaAs at 30kV",
    filepath=emsa[7],
    start_str="#SPECTRUM    : Spectral Data Starts Here",
    stop_str="#ENDOFDATA   : ",
    line_endings="\n",
    delimiter=", ",
    # calibrate on Ga_La and As_Ka
    peaks_keV=[1.098, 10.5436],
    peaks_names=['Ga L&#945;</sub>', 'As K&#945;</sub>'],
    peaks_channel=[131, 1072, 944, 1044],
)

Reading C:\Users\Brynjar\Documents\Masteroppgave\2022-09-06_EDS-Apreo\exports\GaAs_30kV.emsa
The first line looks like this: '#FORMAT      : EMSA/MAS Spectral Data File\n'
Reading from line 42 to 2091.
2048 data points, first entry = [-0.2, 0.0], last entry = [20.27, 52.0]



In [3]:
ga30 = get_multiple_data_arrays(filters=["GaAs_30"])[0][1]

In [4]:
# Ga La, Ga Lb1, As La, As Lb1, Ga Ka, Ga Kb, As Ka, As Kb
peaks = [['Ga', 'La'], ['Ga', 'Lb1'], ['As', 'La'], ['As', 'Lb1'], ['Ga', 'Ka'], ['Ga', 'Kb'], ['As', 'Ka'], ['As', 'Kb']]
peaks_names = ['Ga L&#945;', 'Ga L&#946;<sub>1</sub>', 'As L&#945;', 'As L&#946;<sub>1</sub>', 'Ga K&#945;', 'Ga K&#946;', 'As K&#945;', 'As K&#946;'],
ga_ = hs.material.elements['Ga'].Atomic_properties.Xray_lines.as_dictionary()
as_ = hs.material.elements['As'].Atomic_properties.Xray_lines.as_dictionary()

In [5]:
peaks_keV = []
peaks_weight = []
print(f"{'line':<8} {'energy':<8} {'weight':<6}")

for p in peaks:
    if p[0] == 'Ga':
        print(f"{p[0]:<3} {p[1]:<4} {ga_[p[1]]['energy (keV)']:<8} {ga_[p[1]]['weight']:<6}")
        peaks_keV.append(ga_[p[1]]['energy (keV)'])
        peaks_weight.append(ga_[p[1]]['weight'])
    else:
        print(f'{p[0]:<3} {p[1]:<4} {as_[p[1]]["energy (keV)"]:<8} {as_[p[1]]["weight"]:<6}')
        peaks_keV.append(as_[p[1]]['energy (keV)'])
        peaks_weight.append(as_[p[1]]['weight'])
peaks_keV

line     energy   weight
Ga  La   1.098    1.0   
Ga  Lb1  1.1249   0.16704
As  La   1.2819   1.0   
As  Lb1  1.3174   0.16704
Ga  Ka   9.2517   1.0   
Ga  Kb   10.2642  0.1287
As  Ka   10.5436  1.0   
As  Kb   11.7262  0.14589


[1.098, 1.1249, 1.2819, 1.3174, 9.2517, 10.2642, 10.5436, 11.7262]

In [6]:
# fitting
fit_vals = fit_n_peaks_to_gaussian(
    x=s["channel"],
    y=s["intensity"],
    guessed_peaks=s["peaks_channel"],
)
s["fit_params"] = fit_vals[0]
s["fit_cov"] = fit_vals[1]
s["peaks_channel"] = s["fit_params"][1::3]
s["intensity_fit"] = n_gaussians(s["channel"], *fit_vals[0])
print(f'Fitted peaks at channel: {s["peaks_channel"]}')

Fitted peaks at channel: [ 130.56740439 1072.45815316  943.74174496 1045.33579055]


In [7]:
fig_fit = plotly_plot(
    s["channel"],
    s["intensity"],
    y_fit=s["intensity_fit"],
    vlines=s["peaks_channel"],
    vlines_name=s["peaks_names"],
    title=f"Gaussian fitting of {s['name']}",
)
# fig_fit = plotly_plot(x=s['channel'], fig=fig_fit, fit_params=s['fit_params'], title=f"Gaussian fitting of {s['name']}")
# fig_fit.show()

calib = calibrate_channel_width_two_peaks(s["peaks_channel"], s["peaks_keV"])


The calibration factor is: 0.0100283 keV/channel, with 21.078 channels zero offset


In [8]:
s["dispersion"] = calib[0]
s["offset"] = calib[1]
# s["dispersion"] = 0.010028 # model fit calibration in HyperSpy
# s["offset"] = 0.211377  / 0.010028
s["kev_calibrated"] = (s["channel"] - s["offset"]) * s["dispersion"]

print("Peak name        | Peak position (mu) [keV] | Peak amplitude (a) | FWHM (std*2*sqrt(2*ln(2))) [keV] | Area under peak")
for i in range(len(s["fit_params"]) // 3):
    try:
        peak_name = s["peaks_names"][i]
    except (
        IndexError,
        TypeError,
    ):  # if s['peaks_names'] is None or not as long as the number of peaks
        peak_name = f"peak {i}"
    mu = channel_to_keV(s, value=s['fit_params'][i*3+1])
    amp = s['fit_params']
    fwhm = channel_to_keV(s, value=s['fit_params'][i * 3 + 2] * 2 * (np.log(2) * 2)**0.5, use_offset=False)
    area = area_under_peak(s['fit_params'][i*3+1], s['fit_params'][2 + 3 * i], s['fit_params'][3 * i])
    print(f"{peak_name:<16} | {mu:<24.6f} | {amp[i*3]:<18.4f} | {fwhm:<32.4f} | {area:.6f}")


s["peaks_keV_fitted"] = channel_to_keV(s, value=s['peaks_channel'])



Peak name        | Peak position (mu) [keV] | Peak amplitude (a) | FWHM (std*2*sqrt(2*ln(2))) [keV] | Area under peak
Ga L&#945;</sub> | 1.098000                 | 0.9979             | 0.0750                           | 0.995223
As K&#945;</sub> | 10.543600                | 0.2013             | 0.1724                           | 0.200806
peak 2           | 9.252788                 | 0.3561             | 0.1633                           | 0.355143
peak 3           | 10.271608                | 0.0534             | 0.2027                           | 0.053281


In [9]:
channels = np.arange(0, 2048)

def make_fit_y_vals(order=12, prominence=0.01):
    fit_vals, _ = make_fit(s['intensity'], channels, deg=order, prominence=prominence)
    bg_fit = np.polyval(fit_vals[:order+1], channels)
    fit = n_gaussians_and_m_order_bg(channels, order, *fit_vals)
    print(f'rms error for order {order}: {np.sqrt(np.mean((s["intensity"] - fit)**2)):.5f}')
    return fit, bg_fit

f12, bg12 = make_fit_y_vals(order=12, prominence=0.01)

rms error for order 12: 0.00140


In [25]:
fig = go.Figure()
fig.update_layout(yaxis_range=[-0.01,0.4], xaxis_range=[9.15,9.45], xaxis_title="Energy [keV]", yaxis_title="Intensity [a.u.]")
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
fig.update_layout(width=800, height=500)
fig.add_trace(go.Scatter(x=s["kev_calibrated"], y=f12, name="Fit"))
fig.add_trace(go.Scatter(x=s["kev_calibrated"], y=s["intensity"], name="Raw data", mode='markers'))

lines = [
#  ['Fitted peak center', 9.25279, [  118,    0], 'darkred'],
 ['Fitted peak center, i.e. GaAs calibrated K\u03B1', 9.25279, [  181,    0], 'darkred'],

 ['HS K\u03B1 ', 9.2517,        [  85,    0], 'darkblue'],
 ['XRB K\u03B11 ', 9.25174,      [95,    0],  'darkgreen'],
 ['XRB K\u03B12', 9.22482,       [ 158,    0], 'darkgreen'],
 ['AZ calibrated K\u03B1', 9.23741, [  148,    0], 'sienna'],
 ['Mo calibrated K\u03B1', 9.2517+0.0119, [  85,    0], 'darkorange'],
 ['HS calibrated K\u03B1 ', 9.2517+0.0009, [  115,    0], 'grey'],
 ]
fig.update_layout(legend=dict(x=0.83, y=0.97, traceorder="normal", font=dict(size=14, color="black"), bgcolor="LightSteelBlue", bordercolor="Black", borderwidth=2))

for i in range(len(lines)):
    fig.add_shape(type="line", x0=lines[i][1], y0=-0.01, x1=lines[i][1], y1=0.4, line=dict(color=lines[i][3], width=2, dash="dash"), name=lines[i][0],)
y_annotate = [0.02, 0.040, 0.06, 0.08, 0.1, 0.12, 0.14]
for i in range(len(lines)):
    fig.add_annotation(x=lines[i][1], y=y_annotate[i], text=lines[i][0], showarrow=True, arrowhead=0, ax=lines[i][2][0], ay=lines[i][2][1])
annotation = 'Energy difference from fitted peak center<br><br> Peak                  &#916;E   <br>'
for i in range(1, len(lines)):
    annotation += f'{lines[i][0]:>20} {(lines[i][1] - lines[0][1])*1000:>15.2f} eV<br>'
fig.add_annotation(x=0.975, y=0.51, xref='paper', yref='paper', showarrow=False, text=annotation, align='right', borderpad=4)
fig

# set EB Garamond SemiBold font
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=14, color="black"))

fig.write_image("../plots/Ga-K.png", width=800, height=500, scale=2)
fig.write_image("../../tex_project_TFY4520/figures/Ga-K.png", width=800, height=500, scale=2)